In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import linear_model

# Prepare Data for EDA

In [2]:
#load the cleaned lending club data set
df1 = pd.read_csv('Loan1.csv')
df2 = pd.read_csv('Loan2.csv')

In [7]:
df1.head()

,id,loan_status,loan_amnt,term,int_rate,grade,sub_grade,installment,pymnt_plan,annual_inc,...,pub_rec_bankruptcies,revol_bal,revol_util,tax_liens,tot_cur_bal,tot_hi_cred_lim,total_acc,total_bal_ex_mort,total_bal_il,total_bc_limit
0,114133086,Current,18000.0,36,11.99,1,9,597.78,0,40800.0,...,0.0,11022.0,62.6,0.0,73967.0,92443.0,67.0,73967.0,62945.0,12700.0
1,113327093,Current,36000.0,36,16.02,2,14,1266.01,0,98000.0,...,0.0,24455.0,63.5,0.0,37296.0,55621.0,28.0,37296.0,12841.0,17200.0
2,119396821,Current,1200.0,36,16.02,2,14,42.21,0,48000.0,...,0.0,8371.0,85.4,0.0,24227.0,35259.0,24.0,24227.0,15856.0,7800.0
3,113509028,Current,24000.0,60,10.42,1,7,514.91,0,63000.0,...,0.0,6162.0,38.3,0.0,124925.0,196850.0,15.0,6612.0,0.0,13500.0
4,114749248,Current,12000.0,36,7.07,0,1,370.91,0,120000.0,...,0.0,15025.0,26.7,0.0,15720.0,57242.0,12.0,15720.0,0.0,47800.0


In [6]:
df1 = pd.get_dummies(df1, columns=['loan_status'])

In [7]:
df1=df1.drop(['loan_status_Current'], axis=1)

In [8]:
#convert it to csv to do EDA with deplyr in R
df1.to_csv('lending_club_training.csv')

In [9]:
df1.shape

(61072, 48)

# Filter for Term = 36

In [10]:
df1_30 =  df1.loc[df1['term']==36]

In [11]:
df1_30.head()

,id,loan_amnt,term,int_rate,grade,sub_grade,installment,pymnt_plan,annual_inc,emp_length,...,revol_bal,revol_util,tax_liens,tot_cur_bal,tot_hi_cred_lim,total_acc,total_bal_ex_mort,total_bal_il,total_bc_limit,loan_status_Fail
0,114133086,18000.0,36,11.99,1,9,597.78,0,40800.0,0,...,11022.0,62.6,0.0,73967.0,92443.0,67.0,73967.0,62945.0,12700.0,0
1,113327093,36000.0,36,16.02,2,14,1266.01,0,98000.0,10,...,24455.0,63.5,0.0,37296.0,55621.0,28.0,37296.0,12841.0,17200.0,0
2,119396821,1200.0,36,16.02,2,14,42.21,0,48000.0,1,...,8371.0,85.4,0.0,24227.0,35259.0,24.0,24227.0,15856.0,7800.0,0
4,114749248,12000.0,36,7.07,0,1,370.91,0,120000.0,4,...,15025.0,26.7,0.0,15720.0,57242.0,12.0,15720.0,0.0,47800.0,0
6,116807551,2400.0,36,7.35,0,3,74.49,0,55000.0,2,...,15573.0,86.5,0.0,156278.0,183000.0,18.0,22874.0,7301.0,9800.0,0


# x and y for Variable Selection and Training Data 

#! encoded training data sets are accidentially called test and test is called vali later on!

In [12]:
#shuffle data
df1_30 = df1_30.sample(frac=1)

In [13]:
df1_30.shape
df1_30.to_csv('final_test_data_set.csv')

In [14]:
y = df1_30.loan_status_Fail

In [15]:
x= df1_30.drop(['id', 'loan_status_Fail', 'term'], axis=1)

In [16]:
#splitting df1_30 into a sub-dataset to do the feature selection and into a sub-dataset to do the model selection
y_fs = y.iloc[:7500]
y_test = y.iloc[7501:]
x_fs = x.iloc[:7500]
x_test = x.iloc[7501:]


df1_30_test = df1_30.iloc[7501:]

y_test2 = df1_30_test.loan_status_Fail

#grade0
test_grade0 = df1_30_test.loc[df1_30_test['grade']==0]
x_test_grade0 = x_test.loc[x_test['grade']==0]
y_test_grade0 = test_grade0.loan_status_Fail


#grade1
test_grade1 = df1_30_test.loc[df1_30_test['grade']==1]
x_test_grade1 = x_test.loc[x_test['grade']==1]
y_test_grade1 = test_grade1.loan_status_Fail


#grade2
test_grade2 = df1_30_test.loc[df1_30_test['grade']==2]
x_test_grade2 = x_test.loc[x_test['grade']==2]
y_test_grade2 = test_grade2.loan_status_Fail


#grade3
test_grade3 = df1_30_test.loc[df1_30_test['grade']==3]
x_test_grade3 = x_test.loc[x_test['grade']==3]
y_test_grade3 = test_grade3.loan_status_Fail


#grade4
test_grade4 = df1_30_test.loc[df1_30_test['grade']==4]
x_test_grade4 = x_test.loc[x_test['grade']==4]
y_test_grade4 = test_grade4.loan_status_Fail


#grade5
test_grade5 = df1_30_test.loc[df1_30_test['grade']==5]
x_test_grade5 = x_test.loc[x_test['grade']==5]
y_test_grade5 = test_grade5.loan_status_Fail


#grade6
test_grade6 = df1_30_test.loc[df1_30_test['grade']==6]
x_test_grade6 = x_test.loc[x_test['grade']==6]
y_test_grade6 = test_grade6.loan_status_Fail

# Feature Selection (Lasso)

In [17]:
alphas = np.arange(10, 100, 10)
scores = np.empty_like(alphas)
for i,a in enumerate(alphas):
    lasso = linear_model.Lasso()
    lasso.set_params(alpha=a)
    lasso.fit(x_fs, y_fs)
    scores[i] = lasso.score(x_fs, y_fs)
    print(a, lasso.coef_)

10 [ 3.04430126e-06  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -9.48757566e-08 -0.00000000e+00
  0.00000000e+00 -0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.52215831e-06  0.00000000e+00
  3.31537397e-06  0.00000000e+00  0.00000000e+00  0.00000000e+00
 -2.28260132e-06  0.00000000e+00  0.00000000e+00  0.00000000e+00
 -0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  1.15693078e-06  0.00000000e+00  0.00000000e+00  6.69223940e-07
 -5.98151092e-07  0.00000000e+00 -4.79247773e-08 -0.00000000e+00
 -1.00455868e-06]
20 [ 2.78019016e-06  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -9.44014539e-08 -0.00000000e+00
  0.00000000e+00 -0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.39035537e-06  0.00000000e+00
 

In [18]:
lasso2 = linear_model.Lasso()
lasso2.set_params(alpha=60)
lasso2.fit(x_fs, y_fs)

Lasso(alpha=60, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [19]:
list(zip(x, lasso2.coef_))

[('loan_amnt', 1.9519081410503024e-06),
 ('int_rate', 0.0),
 ('grade', 0.0),
 ('sub_grade', 0.0),
 ('installment', 0.0),
 ('pymnt_plan', 0.0),
 ('annual_inc', -6.237772350053021e-08),
 ('emp_length', -0.0),
 ('dti', 0.0),
 ('fico_range_high', -0.0),
 ('verification_status', 0.0),
 ('purpose', 0.0),
 ('home_ownership', 0.0),
 ('application_type', 0.0),
 ('avg_cur_bal', -7.36529522858501e-07),
 ('delinq_2yrs', 0.0),
 ('delinq_amnt', 0.0),
 ('earliest_cr_line', 0.0),
 ('inq_last_12m', 0.0),
 ('inq_last_6mths', 0.0),
 ('max_bal_bc', -0.0),
 ('num_accts_ever_120_pd', 0.0),
 ('num_actv_bc_tl', 0.0),
 ('num_bc_tl', 0.0),
 ('num_il_tl', -0.0),
 ('num_op_rev_tl', 0.0),
 ('num_rev_accts', 0.0),
 ('num_tl_90g_dpd_24m', 0.0),
 ('open_il_12m', 0.0),
 ('open_il_24m', 0.0),
 ('open_act_il', 0.0),
 ('open_rv_12m', 0.0),
 ('open_rv_24m', 0.0),
 ('percent_bc_gt_75', 0.0),
 ('pub_rec', 0.0),
 ('pub_rec_bankruptcies', 0.0),
 ('revol_bal', 7.324955968053954e-07),
 ('revol_util', 0.0),
 ('tax_liens', 0.0),


In [20]:
# x with feature selection
x_s= x_test[['loan_amnt','annual_inc', 'avg_cur_bal','revol_bal', 'tot_cur_bal', 'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_bc_limit' ]]

# Fuction to get Accuracy, Investment Volume, Roi and Profit out of each Probability Model (Training Data)


In [21]:
def prob_to_output_return(prob, cut_off_value, x, y):
    
    #probabilities of not default
    not_default_p=[]
    for e in prob:
        not_default_p.append(e[0])
        
    #get prediction based on cut-off value
    logit_p4=[]
    for e in not_default_p:
        if e > cut_off_value:
            logit_p4.append(0)
        else:
            logit_p4.append(1)
            
    #convert results into array
    logit_p5 = np.asarray(logit_p4)
    
    #match arrays
    log_s1=np.vstack((y, logit_p5))
    log_s1
    
    #convert to python dataframe
    log_s2 = pd.DataFrame(data=log_s1)
    log_s3= log_s2.transpose()
    log_s3.columns = ['outcome', 'prediction']
    
    
    # get total sum loaned
    loan_amount1= pd.concat([log_s3, x[['loan_amnt', 'int_rate']]], axis=1)
    loan_amount2 =  loan_amount1.loc[loan_amount1['prediction']==0]
    loan_amount3= loan_amount2['loan_amnt'].sum()
    
    #get revenue
    revenue1 =  loan_amount2.loc[loan_amount2['outcome']==0]
    revenue1['revenue'] = (revenue1.loan_amnt*((1+(revenue1.int_rate/100))**3))-revenue1.loan_amnt
    revenue= revenue1['revenue'].sum()

    
    #get loss
    loss1 =  loan_amount2.loc[loan_amount2['outcome']==1]
    loss2 =  (loss1['loan_amnt'].sum()) 
    
    #real profit
    real_profit=revenue-loss2
    
    #calculate return output
    np.seterr(all='ignore')
    miss = loan_amount2['outcome'].sum()
    accuracy = (len(loan_amount2.index)-miss)/len(loan_amount2.index)
    default_rate = 1-accuracy
    volume = loan_amount3
    annual_roi =(1+((revenue-loss2)/loan_amount3))**0.33
    profit = revenue-loss2
    return(accuracy, volume, profit, annual_roi, default_rate)
    


# Get cutoff value that max profits (Training Data)

In [22]:
def profit_maximalising_cuttoff_value(prob,x,y):
#test for a range of values
    results=[]
    for e in range(1, 101, 1):
        results.append(prob_to_output_return(prob, (e/100),x, y ))

    #get output into lists in order to plot    
    acc=[]
    vol=[]
    access_roi=[]
    profit=[]
    roi=[]
    de=[]
    for e in results:
        acc.append(e[0])
        vol.append(e[1])
        profit.append(e[2])
        roi.append(e[3])
        de.append(e[4])    
    
    print('cutoff value:',profit.index(max(profit)), '      Investment Amount:',vol[profit.index(max(profit))],'      profit:', round(max(profit),2), '      roi:',roi[profit.index(max(profit))]   )
    
    

In [23]:
def profit_maximalising_print(prob):
#test for a range of values
    results=[]
    for e in range(0, 100, 1):
        results.append(prob_to_output_return(prob, (e/100)))
    

# Visualisation

In [24]:
def plot(prob,x,y):
#test for a range of values
    results=[]
    for e in range(0, 100, 1):
        results.append(prob_to_output_return(prob, (e/100),x, y ))

    #get output into lists in order to plot    
    acc=[]
    vol=[]
    access_roi=[]
    profit=[]
    roi=[]
    de=[]
    for e in results:
        acc.append(e[0])
        vol.append(e[1])
        profit.append(e[2])
        roi.append(e[3])
        de.append(e[4]) 
        
    plt.plot(profit)
    plt.axis([0,100,0,100000000])
    plt.title('Profit')
    plt.show()
    
    plt.plot(roi)
    plt.axis([0,100,1.05,1.15])
    plt.title('Return on Investment')   
    plt.show()
    
    plt.plot(vol)
    plt.axis([0,100, 0, 500000000])
    plt.title('Investment Volume')  
    plt.show()
    
    plt.plot(de)
    plt.axis([0,100,0,0.2])
    plt.title('Default Rate')      
    plt.show()

# LDA (Training Data)

In [25]:
# linear discriminate analysis full model
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
lda_p = cross_val_predict(lda, x_test, y_test, cv=10, method='predict_proba')
lda_p

array([[0.84174182, 0.15825818],
       [0.93804252, 0.06195748],
       [0.93793241, 0.06206759],
       ...,
       [0.7631624 , 0.2368376 ],
       [0.93491643, 0.06508357],
       [0.95460698, 0.04539302]])

In [29]:
# linear discriminate analysis with feature selection
lda2 = LinearDiscriminantAnalysis()
lda_p2 = cross_val_predict(lda, x_s, y_test, cv=10, method='predict_proba')
profit_maximalising_cuttoff_value(lda_p2, x_test,y_test)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


cutoff value: 76       Investment Amount: 258683300.0       profit: 54855391.05       roi: 1.0655222106782265


# Logit (Training Data)

In [31]:
# logit full model
from sklearn.linear_model import LogisticRegression
logit = LogisticRegression()
logit_p = cross_val_predict(logit, x_test, y_test, cv=10, method='predict_proba')
profit_maximalising_cuttoff_value(logit_p, x_test, y_test)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


cutoff value: 75       Investment Amount: 251681275.0       profit: 56005857.65       roi: 1.0685510653545192


In [39]:
# logit reduced model
from sklearn.linear_model import LogisticRegression
logit_p2 = cross_val_predict(logit, x_s, y_test, cv=10, method='predict_proba')
profit_maximalising_cuttoff_value(logit_p2, x_test, y_test)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


cutoff value: 59       Investment Amount: 260102100.0       profit: 54867017.56       roi: 1.0651995099837654


# Gradient Boosting (Training Data)

In [28]:
from sklearn import ensemble
gbr = ensemble.GradientBoostingClassifier()
gbr_p = cross_val_predict(gbr, x_test, y_test, cv=10, method='predict_proba')
profit_maximalising_cuttoff_value(gbr_p, x_test, y_test)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


cutoff value: 68       Investment Amount: 245688275.0       profit: 58768982.14       roi: 1.0733387947909803


In [29]:
from sklearn import ensemble
gbr_p2 = cross_val_predict(gbr, x_s, y_test, cv=10, method='predict_proba')
profit_maximalising_cuttoff_value(gbr_p2, x_test, y_test)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


cutoff value: 67       Investment Amount: 259093025.0       profit: 56674848.35       roi: 1.0674585811608455


# Random Forrest (Training Data)

In [43]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier( max_depth=4,random_state=0)
rfc_p = cross_val_predict(rfc, x_test, y_test, cv=10, method='predict_proba')
profit_maximalising_cuttoff_value(rfc_p,x_test, y_test)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


cutoff value: 73       Investment Amount: 252303325.0       profit: 56463596.87       roi: 1.0689159859849693


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:49: RuntimeWarning: invalid value encountered in double_scalars
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:52: RuntimeWarning: invalid value encountered in double_scalars


In [48]:
rfc = RandomForestClassifier( max_depth=4,random_state=0)
rfc_p = cross_val_predict(rfc, x_s, y_test, cv=10, method='predict_proba')
profit_maximalising_cuttoff_value(rfc_p,x_test, y_test)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:49: RuntimeWarning: invalid value encountered in double_scalars
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:52: RuntimeWarning: invalid value encountered in double_scalars


cutoff value: 76       Investment Amount: 259998725.0       profit: 54874428.0       roi: 1.065232128825325


# Test (out-of-sample)

In [30]:
#data set pre prepartation
df2_30 =  df2[df2['term']==36]
df2_30 = pd.get_dummies(df2_30, columns=['loan_status'])
df2_30=df2_30.drop(['loan_status_Current'], axis=1)
y_t = df2_30.loan_status_Fail
x_t =df2_30.drop(['id', 'loan_status_Fail', 'term'], axis=1)

In [51]:
lda_vali = LinearDiscriminantAnalysis()
lda_vali.fit(x,y)

LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
              solver='svd', store_covariance=False, tol=0.0001)

In [ ]:
#grade0
vali_grade0 = df2_30[df2_30['grade']==0]
x_vali_grade0 = x_t[x_t['grade']==0]
y_vali_grade0 = vali_grade0.loan_status_Fail


#grade1
vali_grade1 = df2_30[df2_30['grade']==1]
x_vali_grade1 = x_t[x_t['grade']==1]
y_vali_grade1 = vali_grade1.loan_status_Fail


#grade2
vali_grade2 = df2_30[df2_30['grade']==2]
x_vali_grade2 = x_t[x_t['grade']==2]
y_vali_grade2 = vali_grade2.loan_status_Fail


#grade3
vali_grade3 = df2_30[df2_30['grade']==3]
x_vali_grade3 = x_t[x_t['grade']==3]
y_vali_grade3 = vali_grade3.loan_status_Fail


#grade4
vali_grade4= df2_30[df2_30['grade']==4]
x_vali_grade4 = x_t[x_t['grade']==4]
y_vali_grade4 = vali_grade4.loan_status_Fail

#grade5
vali_grade5= df2_30[df2_30['grade']==5]
x_vali_grade5 = x_t[x_t['grade']==5]
y_vali_grade5 = vali_grade5.loan_status_Fail

#grade6
vali_grade6= df2_30[df2_30['grade']==6]
x_vali_grade6= x_t[x_t['grade']==6]
y_vali_grade6 = vali_grade6.loan_status_Fail

In [48]:
#startegy 1
gbr_vali = ensemble.GradientBoostingClassifier()
gbr_vali.fit(x,y)
vali_gbr = gbr_vali.predict_proba(x_t)
prob_to_output_return(vali_gbr, 0.68, x_t, y_t)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(0.9098674756506466,
 339591700.0,
 87528891.94517022,
 1.078613461350688,
 0.09013252434935337)

In [53]:
#startegy 2
prob_to_output_return(vali_gbr, 0.9, x_t, y_t)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(0.9559516714525654,
 166613675.0,
 53118658.819463566,
 1.0956213766139131,
 0.04404832854743457)

In [59]:

#strategy 3
prob_to_output_return(vali_gbr, 0.95, x_t, y_t)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(0.9757295579312337,
 62222850.0,
 21660283.0007097,
 1.1035935800112686,
 0.024270442068766274)

In [55]:
#prepare data to plot in R
    results=[]
    for e in range(1, 101, 1):
        results.append(prob_to_output_return(vali_gbr, (e/100),x_t, y_t ))

    #get output into lists in order to plot    
    acc=[]
    vol=[]
    access_roi=[]
    profit=[]
    roi=[]
    de=[]
    for e in results:
        acc.append(e[0])
        vol.append(e[1])
        profit.append(e[2])
        roi.append(e[3])
        de.append(e[4])  

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [57]:
#prepare data to plot in R
data_tuples2 = list(zip(a, acc,vol, profit, roi, de))
aa=pd.DataFrame(data_tuples2, columns=['cutoff','accuracy', 'volumne', 'profit', 'roi', 'default_rate'])
aa.to_csv('Gradient_boosting_test.csv')